In [83]:
import os
from google.cloud import bigquery
import pandas as pd
from get_sheets import get_sheets

In [80]:
sheet = get_sheets('reporting.json', 'REPORTING', f'Report_shopping_apparel_abahouse') 

In [170]:
df = pd.DataFrame(sheet.get_all_records())

In [169]:
project_id = "glass-world-403808"
dataset_id = "scrapingdata"
table_id = 'scrapingtry'
client = bigquery.Client()

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\HP\Documents\ARI\Github\GitWebSheets\access_bq_scraping.json"

# from pandas.io import gbq
temp_df = df.copy(deep=True).replace('','null')
temp_df[['lat', 'lon']] = temp_df[['lat', 'lon']].astype(str)
temp_df = temp_df[['url_store', 'store_name', 'brand', 'address', 'tel_no',
                   'lat', 'lon', 'open_hours', 'holiday', 'parking', 'smoking',
                   'additional_info1', 'additional_info2', 'scrape_date']]

temp_df.to_gbq(destination_table=f'{dataset_id}.{table_id}',
          project_id=project_id,
          if_exists='append')


100%|██████████| 1/1 [00:00<?, ?it/s]


In [ ]:
# PYTHON PANDAS METHOD
table_ref = client.dataset(dataset_id).table(table_id)

try:
    query_job = client.load_table_from_dataframe(df, table_ref)
    query_job.result()
except:
    query_job = client.create_table(table_ref)
    query_job = client.load_table_from_dataframe(df, table_ref)
    query_job.result()

In [ ]:
# CLASSIC METHOD

def load_gcs_to_bigquery_event_data(GCS_URI, TABLE_ID, table_schema):
    job_config = bigquery.LoadJobConfig(
        schema=table_schema,
        source_format=bigquery.SourceFormat.CSV,
        write_disposition = 'WRITE_TRUNCATE',
        max_bad_records=100,
        )
    job_config.skip_leading_rows = 1
    for row in GCS_URI:
        load_job = client.load_table_from_file(
            row, TABLE_ID, job_config=job_config
        )
    
    load_job.result()
    table = client.get_table(TABLE_ID)

    print("Loaded {} rows to table {}".format(table.num_rows, TABLE_ID))

GCS_URI = csv_df
full_table_id = "{}.{}.{}".format(project_id, dataset_id, table_id)
client = bigquery.Client()

table_schema = list(df.columns)
bigquery_table_schema = [bigquery.SchemaField(x, "STRING") if x != 'scrape_date' and x !='registration_date' else bigquery.SchemaField(x, "DATE") for x in table_schema]

# if __name__ == '__main__':
#     load_gcs_to_bigquery_event_data(GCS_URI, full_table_id, bigquery_table_schema)


# EXAMPLE QUERY METHOD
# data_query = f"""
#         CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.{table_id}` AS
#         SELECT *
#         FROM `{project_id}.{dataset_id}.{table_id}`;
#     """

# query_job = client.query(data_query)
# query_job.result()